В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [ ]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [ ]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [ ]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [32]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

NameError: name 'w2v_model' is not defined

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [11]:
!pip install gensim

import gensim
import gensim.downloader as api
model = api.load('glove-wiki-gigaword-100')

2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [12]:
def find_similar_words(word):
    try:
        similar_words = model.most_similar(word, topn=10)
        return similar_words
    except:
        return f"Не удалось найти похожие слова для '{word}'"

result = find_similar_words(input("Введите любое слово:"))
print(result)

Введите любое слово:pug
[('rottweiler', 0.6888886094093323), ('puppy', 0.6259522438049316), ('dachshund', 0.5998451709747314), ('raccoon', 0.5800819396972656), ('duckling', 0.5726965665817261), ('pekingese', 0.563757598400116), ('doberman', 0.5511077046394348), ('poodle', 0.5415231585502625), ('skunk', 0.5371769070625305), ('bichon', 0.535858690738678)]


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [21]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [28]:
# Учим модель
model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1
)

# Делаем проверку
print("Похожие на 'варить':")
for word, score in model.wv.most_similar('варить', topn=3):
    print(f"  {word}: {score:.3f}")


Похожие на 'варить':
  вино: 0.240
  ингредиенты: 0.217
  хлеб: 0.194


In [29]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [30]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [33]:
# Найдите слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [61]:
from gensim.models import FastText
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1,
    min_n=2,
    max_n=4
)

print("Модель FastText успешно обучена!")

Модель FastText успешно обучена!


6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [45]:
# Найдите слова, похожие на "варить"
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

# Найдите слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Найдите слова, похожие на "овощи"
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711
Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [62]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Тестируем 3 слова с опечатками на кулинарную тематику
test_words_with_typos = ['фрукды', 'хлеп', 'суб']

for word in test_words_with_typos:
    compare_models(word)


Сравнение для слова: 'фрукды'
  Word2Vec: слово не найдено
  FastText: ['кипятить', 'говядина']

Сравнение для слова: 'хлеп'
  Word2Vec: слово не найдено
  FastText: ['хлеб', 'овощи']

Сравнение для слова: 'суб'
  Word2Vec: слово не найдено
  FastText: ['суп', 'хлеб']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [67]:
# Создаем размеченные документы
from gensim.models.doc2vec import TaggedDocument
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")


Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [70]:
# Обучаем Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [71]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [72]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [73]:
# Сравниваем схожесть документов
def compare_documents(doc2_id, doc4_id):
    similarity = doc_model.dv.similarity(f"doc_{doc2_id}", f"doc_{doc4_id}")
    print(f"Схожесть doc_{doc2_id} и doc_{doc4_id}: {similarity:.4f}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")
    print(f"  doc_{doc4_id}: {documents[doc4_id]}")

compare_documents(2, 4)


Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [74]:
similar_docs = doc_model.dv.most_similar("doc_1", topn=1)
print("\nДокументы, похожие на doc_1:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")


Документы, похожие на doc_1:
  doc_0: 0.2735
    Текст: machine learning is interesting


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [77]:
dritatuschki_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)
try:
    similar = dritatuschki_model.wv.most_similar('горох', topn=5)
    print("\n Слова, похожие на 'горох':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("\n Слово 'горох' не найдено в словаре")

try:
    similar = dritatuschki_model.wv.most_similar('молоко', topn=5)
    print("\n Слова, похожие на 'молоко':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("\n Слово 'молоко' не найдено в словаре")

try:
    similar = dritatuschki_model.wv.most_similar('картофель', topn=5)
    print("\n Слова, похожие на 'картофель':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("\n Слово 'картофель' не найдено в словаре")


 Слова, похожие на 'горох':
  паста: 0.3479
  дрожжи: 0.3220
  чай: 0.3003
  яичница: 0.2837
  мука: 0.2551

 Слова, похожие на 'молоко':
  десерт: 0.4010
  яблоки: 0.3866
  уголь: 0.3247
  хлеб: 0.3031
  начинка: 0.2786

 Слова, похожие на 'картофель':
  десерт: 0.3268
  кипятить: 0.2861
  уголь: 0.2839
  ингредиенты: 0.2775
  вино: 0.2583
